In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import yfinance as yf

stock_urls = [
    "https://www.screener.in/company/ASIANPAINT/",
    "https://www.screener.in/company/RELIANCE/",
    "https://www.screener.in/company/TCS/",
    "https://www.screener.in/company/INFY/",
    "https://www.screener.in/company/HDFCBANK/",
    "https://www.screener.in/company/ICICIBANK/",
    "https://www.screener.in/company/ITC/",
    "https://www.screener.in/company/HINDUNILVR/",
    "https://www.screener.in/company/SBIN/",
    "https://www.screener.in/company/BHARTIARTL/"
]

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
}

data = []

for stock_url in stock_urls:
    response = requests.get(stock_url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        try:
            stock_name = soup.find('h1', class_='h2 shrink-text').get_text(strip=True)
            
            ratios = {}
            ratios_list = soup.find('ul', id='top-ratios').find_all('li')
            for item in ratios_list:
                ratio_name = item.find('span', class_='name').text.strip()
                ratio_value = item.find('span', class_='nowrap value').text.strip()
                ratios[ratio_name] = ratio_value
            
            stock_data = {'Company Name': stock_name}
            stock_data.update(ratios)
            data.append(stock_data)
        except AttributeError:
            print(f"Failed to scrape data for {stock_url}")
    else:
        print(f"Failed to fetch URL: {stock_url} with status code {response.status_code}")

df = pd.DataFrame(data)
df = df.applymap(lambda x: x.replace("\n", "").replace("        ", "").strip() if isinstance(x, str) else x)

symbols = {
    "Asian Paints Ltd": "ASIANPAINT.NS",
    "Reliance Industries Ltd": "RELIANCE.NS",
    "Tata Consultancy Services Ltd": "TCS.NS",
    "Infosys Ltd": "INFY.NS",
    "HDFC Bank Ltd": "HDFCBANK.NS",
    "ICICI Bank Ltd": "ICICIBANK.NS",
    "ITC Ltd": "ITC.NS",
    "Hindustan Unilever Ltd": "HINDUNILVR.NS",
    "State Bank of India": "SBIN.NS",
    "Bharti Airtel Ltd": "BHARTIARTL.NS"
}

def calculate_returns(symbol):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365 * 5)  
    data = yf.download(symbol, start=start_date, end=end_date)
    
    if data.empty:
        return None, None, None  
    
    close_prices = data["Adj Close"]
    latest_price = close_prices.iloc[-1]
    
    returns_6m = returns_1y = returns_5y = None

    if len(close_prices) > 126:
        returns_6m = ((latest_price - close_prices.iloc[-126]) / close_prices.iloc[-126]) * 100
    if len(close_prices) > 252:
        returns_1y = ((latest_price - close_prices.iloc[-252]) / close_prices.iloc[-252]) * 100
    if len(close_prices) > 0:
        returns_5y = ((latest_price - close_prices.iloc[0]) / close_prices.iloc[0]) * 100

    return (
        float(returns_6m) if returns_6m is not None else None,
        float(returns_1y) if returns_1y is not None else None,
        float(returns_5y) if returns_5y is not None else None
    )

returns_data = []

for company, symbol in symbols.items():
    print(f"Fetching data for {company}...")
    returns = calculate_returns(symbol)
    if returns:
        returns_data.append({
            "Company Name": company,
            "6M Returns": returns[0],
            "1Y Returns": returns[1],
            "5Y Returns": returns[2]
        })

returns_df = pd.DataFrame(returns_data)
final_df = pd.merge(df, returns_df, on="Company Name")

print(final_df)
final_df.to_csv("nifty50_stock_data_with_returns.csv", index=False)


C:\Users\dubey\AppData\Local\Temp\ipykernel_29536\2606303267.py:50: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace("\n", "").replace("        ", "").strip() if isinstance(x, str) else x)


Fetching data for Asian Paints Ltd...


[*********************100%***********************]  1 of 1 completed
C:\Users\dubey\AppData\Local\Temp\ipykernel_29536\2606303267.py:86: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(returns_6m) if returns_6m is not None else None,
C:\Users\dubey\AppData\Local\Temp\ipykernel_29536\2606303267.py:87: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(returns_1y) if returns_1y is not None else None,
C:\Users\dubey\AppData\Local\Temp\ipykernel_29536\2606303267.py:88: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(returns_5y) if returns_5y is not None else None


Fetching data for Reliance Industries Ltd...


[*********************100%***********************]  1 of 1 completed
C:\Users\dubey\AppData\Local\Temp\ipykernel_29536\2606303267.py:86: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(returns_6m) if returns_6m is not None else None,
C:\Users\dubey\AppData\Local\Temp\ipykernel_29536\2606303267.py:87: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(returns_1y) if returns_1y is not None else None,
C:\Users\dubey\AppData\Local\Temp\ipykernel_29536\2606303267.py:88: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(returns_5y) if returns_5y is not None else None


...

[*********************100%***********************]  1 of 1 completed
C:\Users\dubey\AppData\Local\Temp\ipykernel_29536\2606303267.py:86: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(returns_6m) if returns_6m is not None else None,
C:\Users\dubey\AppData\Local\Temp\ipykernel_29536\2606303267.py:87: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(returns_1y) if returns_1y is not None else None,
C:\Users\dubey\AppData\Local\Temp\ipykernel_29536\2606303267.py:88: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(returns_5y) if returns_5y is not None else None


Fetching data for Bharti Airtel Ltd...


[*********************100%***********************]  1 of 1 completed

                    Company Name       Market Cap Current Price  \
0               Asian Paints Ltd   ₹2,30,941  Cr.        ₹2,408   
1        Reliance Industries Ltd  ₹17,22,468  Cr.        ₹1,273   
2  Tata Consultancy Services Ltd  ₹16,18,696  Cr.        ₹4,474   
3                    Infosys Ltd   ₹8,30,325  Cr.        ₹2,000   
4                  HDFC Bank Ltd  ₹14,30,929  Cr.        ₹1,872   
5                 ICICI Bank Ltd   ₹9,49,165  Cr.        ₹1,345   
6                        ITC Ltd   ₹5,88,008  Cr.          ₹470   
7         Hindustan Unilever Ltd   ₹5,61,576  Cr.        ₹2,390   
8            State Bank of India   ₹7,68,901  Cr.          ₹862   
9              Bharti Airtel Ltd  ₹10,07,452  Cr.        ₹1,682   

        High / Low Stock P/E Book Value Dividend Yield     ROCE      ROE  \
0  ₹ 3,423 / 2,354      50.3       ₹185        1.38  %  39.5  %  31.3  %   
1  ₹ 1,609 / 1,217      47.2       ₹387        0.39  %  9.60  %  8.42  %   
2  ₹ 4,592 / 3,592      33.7      


C:\Users\dubey\AppData\Local\Temp\ipykernel_29536\2606303267.py:86: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(returns_6m) if returns_6m is not None else None,
C:\Users\dubey\AppData\Local\Temp\ipykernel_29536\2606303267.py:87: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(returns_1y) if returns_1y is not None else None,
C:\Users\dubey\AppData\Local\Temp\ipykernel_29536\2606303267.py:88: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(returns_5y) if returns_5y is not None else None


In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import yfinance as yf

stock_urls = [
    "https://www.screener.in/company/ASIANPAINT/",
    "https://www.screener.in/company/RELIANCE/",
    "https://www.screener.in/company/TCS/",
    "https://www.screener.in/company/INFY/",
    "https://www.screener.in/company/HDFCBANK/",
    "https://www.screener.in/company/ICICIBANK/",
    "https://www.screener.in/company/ITC/",
    "https://www.screener.in/company/HINDUNILVR/",
    "https://www.screener.in/company/SBIN/",
    "https://www.screener.in/company/BHARTIARTL/"
]

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
}

data = []

for stock_url in stock_urls:
    response = requests.get(stock_url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        try:
            stock_name = soup.find('h1', class_='h2 shrink-text').get_text(strip=True)
            
            ratios = {}
            ratios_list = soup.find('ul', id='top-ratios').find_all('li')
            for item in ratios_list:
                ratio_name = item.find('span', class_='name').text.strip()
                ratio_value = item.find('span', class_='nowrap value').text.strip()
                ratios[ratio_name] = ratio_value
            
            stock_data = {'Company Name': stock_name}
            stock_data.update(ratios)
            data.append(stock_data)
        except AttributeError:
            print(f"Failed to scrape data for {stock_url}")
    else:
        print(f"Failed to fetch URL: {stock_url} with status code {response.status_code}")

df = pd.DataFrame(data)
df = df.applymap(lambda x: x.replace("\n", "").replace("        ", "").strip() if isinstance(x, str) else x)
df = df.set_index("Company Name", inplace = False)

print(df)
df.to_csv("stock_data.csv", index=False)




nifty50_symbols = {
    "Asian Paints Ltd": "ASIANPAINT.NS",
    "Reliance Industries Ltd": "RELIANCE.NS",
    "Tata Consultancy Services Ltd": "TCS.NS",
    "Infosys Ltd": "INFY.NS",
    "HDFC Bank Ltd": "HDFCBANK.NS",
    "ICICI Bank Ltd": "ICICIBANK.NS",
    "ITC Ltd": "ITC.NS",
    "Hindustan Unilever Ltd": "HINDUNILVR.NS",
    "State Bank of India": "SBIN.NS",
    "Bharti Airtel Ltd": "BHARTIARTL.NS"
}


def calculate_returns(symbol):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365 * 5)  # Fetch data for the last 5 years
    data = yf.download(symbol, start=start_date, end=end_date)
    
    # Calculate returns based on 'Adj Close' prices
    if data.empty:
        return None, None, None  # Return None if no data available
    
    close_prices = data["Adj Close"]
    latest_price = close_prices.iloc[-1]

    # Get historical prices
    returns_6m = ((latest_price - close_prices.iloc[-126]) / close_prices.iloc[-126]) * 100 if len(close_prices) > 126 else None
    returns_1y = ((latest_price - close_prices.iloc[-252]) / close_prices.iloc[-252]) * 100 if len(close_prices) > 252 else None
    returns_5y = ((latest_price - close_prices.iloc[0]) / close_prices.iloc[0]) * 100 if len(close_prices) > 0 else None

    return returns_6m, returns_1y, returns_5y

# Create an empty list to store return data
returns_data = []

# Calculate returns for each stock
for company, symbol in nifty50_symbols.items():
    print(f"Fetching data for {company}...")
    returns = calculate_returns(symbol)
    returns_data.append({
        "Company Name": company, 
        "6M Returns": returns[0],
        "1Y Returns": returns[1],
        "5Y Returns": returns[2]
    })

# Create a DataFrame for returns
returns_df = pd.DataFrame(returns_data)


stock_df = df

# Merge the stock data with returns
final_df = pd.merge(stock_df, returns_df, on = "Company Name")

# Set 'Company Name' as index
#final_df.set_index("Company Name", inplace=True)

# Display final DataFrame
print(final_df)

# Save to CSV
final_df.to_csv("nifty50_stock_data_with_returns.csv")


C:\Users\dubey\AppData\Local\Temp\ipykernel_29536\642461417.py:49: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace("\n", "").replace("        ", "").strip() if isinstance(x, str) else x)


                                    Market Cap Current Price       High / Low  \
Company Name                                                                    
Asian Paints Ltd                ₹2,30,941  Cr.        ₹2,408  ₹ 3,423 / 2,354   
Reliance Industries Ltd        ₹17,22,468  Cr.        ₹1,273  ₹ 1,609 / 1,217   
Tata Consultancy Services Ltd  ₹16,18,696  Cr.        ₹4,474  ₹ 4,592 / 3,592   
Infosys Ltd                     ₹8,30,325  Cr.        ₹2,000  ₹ 2,007 / 1,358   
HDFC Bank Ltd                  ₹14,30,929  Cr.        ₹1,872  ₹ 1,880 / 1,363   
ICICI Bank Ltd                  ₹9,49,165  Cr.        ₹1,345    ₹ 1,362 / 970   
ITC Ltd                         ₹5,88,008  Cr.          ₹470      ₹ 529 / 399   
Hindustan Unilever Ltd          ₹5,61,576  Cr.        ₹2,390  ₹ 3,035 / 2,170   
State Bank of India             ₹7,68,901  Cr.          ₹862      ₹ 912 / 601   
Bharti Airtel Ltd              ₹10,07,452  Cr.        ₹1,682    ₹ 1,779 / 960   

                           

[*********************100%***********************]  1 of 1 completed


Fetching data for Reliance Industries Ltd...


[*********************100%***********************]  1 of 1 completed


...

Fetching data for State Bank of India...


[*********************100%***********************]  1 of 1 completed


Fetching data for Bharti Airtel Ltd...


[*********************100%***********************]  1 of 1 completed

                    Company Name       Market Cap Current Price  \
0               Asian Paints Ltd   ₹2,30,941  Cr.        ₹2,408   
1        Reliance Industries Ltd  ₹17,22,468  Cr.        ₹1,273   
2  Tata Consultancy Services Ltd  ₹16,18,696  Cr.        ₹4,474   
3                    Infosys Ltd   ₹8,30,325  Cr.        ₹2,000   
4                  HDFC Bank Ltd  ₹14,30,929  Cr.        ₹1,872   
5                 ICICI Bank Ltd   ₹9,49,165  Cr.        ₹1,345   
6                        ITC Ltd   ₹5,88,008  Cr.          ₹470   
7         Hindustan Unilever Ltd   ₹5,61,576  Cr.        ₹2,390   
8            State Bank of India   ₹7,68,901  Cr.          ₹862   
9              Bharti Airtel Ltd  ₹10,07,452  Cr.        ₹1,682   

        High / Low Stock P/E Book Value Dividend Yield     ROCE      ROE  \
0  ₹ 3,423 / 2,354      50.3       ₹185        1.38  %  39.5  %  31.3  %   
1  ₹ 1,609 / 1,217      47.2       ₹387        0.39  %  9.60  %  8.42  %   
2  ₹ 4,592 / 3,592      33.7      

In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import yfinance as yf

stock_urls = [
    "https://www.screener.in/company/ASIANPAINT/",
    "https://www.screener.in/company/RELIANCE/",
    "https://www.screener.in/company/TCS/",
    "https://www.screener.in/company/INFY/",
    "https://www.screener.in/company/HDFCBANK/",
    "https://www.screener.in/company/ICICIBANK/",
    "https://www.screener.in/company/ITC/",
    "https://www.screener.in/company/HINDUNILVR/",
    "https://www.screener.in/company/SBIN/",
    "https://www.screener.in/company/BHARTIARTL/"
]

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
}

data = []

# Scrape stock data
for stock_url in stock_urls:
    response = requests.get(stock_url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        try:
            stock_name = soup.find('h1', class_='h2 shrink-text').get_text(strip=True)
            
            ratios = {}
            ratios_list = soup.find('ul', id='top-ratios').find_all('li')
            for item in ratios_list:
                ratio_name = item.find('span', class_='name').text.strip()
                ratio_value = item.find('span', class_='nowrap value').text.strip()
                ratios[ratio_name] = ratio_value
            
            stock_data = {'Company Name': stock_name}
            stock_data.update(ratios)
            data.append(stock_data)
        except AttributeError:
            print(f"Failed to scrape data for {stock_url}")
    else:
        print(f"Failed to fetch URL: {stock_url} with status code {response.status_code}")

df = pd.DataFrame(data)
df = df.applymap(lambda x: x.replace("\n", "").replace("        ", "").strip() if isinstance(x, str) else x)

# Ticker mapping for Nifty50 stocks
nifty50_symbols = {
    "Asian Paints Ltd": "ASIANPAINT.NS",
    "Reliance Industries Ltd": "RELIANCE.NS",
    "Tata Consultancy Services Ltd": "TCS.NS",
    "Infosys Ltd": "INFY.NS",
    "HDFC Bank Ltd": "HDFCBANK.NS",
    "ICICI Bank Ltd": "ICICIBANK.NS",
    "ITC Ltd": "ITC.NS",
    "Hindustan Unilever Ltd": "HINDUNILVR.NS",
    "State Bank of India": "SBIN.NS",
    "Bharti Airtel Ltd": "BHARTIARTL.NS"
}

# Function to calculate returns
def calculate_returns(symbol):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365 * 5)  # Fetch data for the last 5 years
    data = yf.download(symbol, start=start_date, end=end_date)
    
    # Calculate returns based on 'Adj Close' prices
    if data.empty:
        return None, None, None  # Return None if no data available
    
    close_prices = data["Adj Close"]
    latest_price = close_prices.iloc[-1]

    # Get historical prices
    returns_6m = ((latest_price - close_prices.iloc[-126]) / close_prices.iloc[-126]) * 100 if len(close_prices) > 126 else None
    returns_1y = ((latest_price - close_prices.iloc[-252]) / close_prices.iloc[-252]) * 100 if len(close_prices) > 252 else None
    returns_5y = ((latest_price - close_prices.iloc[0]) / close_prices.iloc[0]) * 100 if len(close_prices) > 0 else None

    return returns_6m, returns_1y, returns_5y

# Create an empty list to store return data
returns_data = []

# Calculate returns for each stock
for company, symbol in nifty50_symbols.items():
    print(f"Fetching data for {company}...")
    returns = calculate_returns(symbol)
    returns_data.append({
        "Company Name": company, 
        "6M Returns": returns[0],
        "1Y Returns": returns[1],
        "5Y Returns": returns[2]
    })

# Create a DataFrame for returns
returns_df = pd.DataFrame(returns_data)

# Merge the stock data with returns
final_df = pd.merge(df, returns_df, on="Company Name")

# Display final DataFrame
print(final_df)

# Save to CSV
final_df.to_csv("nifty50_stock_data_with_returns.csv", index=False)


C:\Users\dubey\AppData\Local\Temp\ipykernel_29536\2415104845.py:50: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace("\n", "").replace("        ", "").strip() if isinstance(x, str) else x)


Fetching data for Asian Paints Ltd...


[*********************100%***********************]  1 of 1 completed


Fetching data for Reliance Industries Ltd...


[*********************100%***********************]  1 of 1 completed


...

Fetching data for State Bank of India...


[*********************100%***********************]  1 of 1 completed


Fetching data for Bharti Airtel Ltd...


[*********************100%***********************]  1 of 1 completed

                    Company Name       Market Cap Current Price  \
0               Asian Paints Ltd   ₹2,30,941  Cr.        ₹2,408   
1        Reliance Industries Ltd  ₹17,22,468  Cr.        ₹1,273   
2  Tata Consultancy Services Ltd  ₹16,18,696  Cr.        ₹4,474   
3                    Infosys Ltd   ₹8,30,325  Cr.        ₹2,000   
4                  HDFC Bank Ltd  ₹14,30,929  Cr.        ₹1,872   
5                 ICICI Bank Ltd   ₹9,49,165  Cr.        ₹1,345   
6                        ITC Ltd   ₹5,88,008  Cr.          ₹470   
7         Hindustan Unilever Ltd   ₹5,61,576  Cr.        ₹2,390   
8            State Bank of India   ₹7,68,901  Cr.          ₹862   
9              Bharti Airtel Ltd  ₹10,07,452  Cr.        ₹1,682   

        High / Low Stock P/E Book Value Dividend Yield     ROCE      ROE  \
0  ₹ 3,423 / 2,354      50.3       ₹185        1.38  %  39.5  %  31.3  %   
1  ₹ 1,609 / 1,217      47.2       ₹387        0.39  %  9.60  %  8.42  %   
2  ₹ 4,592 / 3,592      33.7      

In [1]:
import requests
from bs4 import BeautifulSoup

stock_url = "https://www.screener.in/company/ASIANPAINT/"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
}

response = requests.get(stock_url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

stock_name = soup.find('h1', class_='h2 shrink-text').get_text(strip=True)

ratios = {}
ratios_list = soup.find('ul', id='top-ratios').find_all('li')
for item in ratios_list:
    ratio_name = item.find('span', class_='name').text.strip()
    ratio_value = item.find('span', class_='nowrap value').text.strip()
    ratios[ratio_name] = ratio_value

print(f"Company Name: {stock_name}")
print("Data:")
for key, value in ratios.items():
    print(f"  {key}: {value}")

Company Name: Asian Paints Ltd
Data:
  Market Cap: ₹
        2,30,941
        
          Cr.
  Current Price: ₹
        2,408
  High / Low: ₹ 3,423 / 2,354
  Stock P/E: 50.3
  Book Value: ₹
        185
  Dividend Yield: 1.38
        
          %
  ROCE: 39.5
        
          %
  ROE: 31.3
        
          %
  Face Value: ₹
        1.00
